In [1]:
import pandas as pd
import numpy as np
import json
total_max = pd.read_csv("../public/procressData/distance_max.csv")

In [2]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, metric = "precomputed",learning_rate = 'auto', init='random',random_state=0)
X_tsne = tsne.fit_transform(total_max.values)

In [3]:
import sklearn
print(sklearn.__version__)

1.1.3


In [4]:
tsne_res  = pd.DataFrame(tsne.embedding_, columns=['x', 'y'])
tsne_res['frame'] = total_max.index
with open('../public/procressData/index_frame_dict.json', 'r') as f:
    index_frame_dict = json.load(f)
frame_logInfos = pd.DataFrame.from_dict(index_frame_dict, orient="index")
frame_logInfos["frame"] = frame_logInfos.index
frame_logInfos["frame"] = frame_logInfos["frame"].astype(int)
tsnelogInfos = tsne_res.merge(frame_logInfos, on = ["frame"])

In [5]:
same_frame_infos = pd.read_csv('../public/procressData/same_frame_relationship.csv').groupby(["logId", "same_frame"]).apply(lambda x: x["frame"].tolist()).reset_index()
same_frame_infos = same_frame_infos.rename(columns = {'same_frame':'first_frame', 0:'frames_list'})
same_frame_infos
res = tsnelogInfos.merge(same_frame_infos, left_on = ["logId", "same_frame"], right_on = ["logId","first_frame"])
res.sort_values(by = ["logId", "first_frame"], inplace = True)
res

,x,y,frame,same_frame,logId,first_frame,frames_list
0,24.945734,-0.490164,0,0,02cf0ce1-699a-373b-86c0-eb6fd5f4697a,0,[0]
1,32.912552,3.655365,1,1,02cf0ce1-699a-373b-86c0-eb6fd5f4697a,1,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,36.628139,10.191377,2,32,02cf0ce1-699a-373b-86c0-eb6fd5f4697a,32,"[32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 4..."
3,37.112850,10.791130,3,49,02cf0ce1-699a-373b-86c0-eb6fd5f4697a,49,"[49, 50, 51, 52, 53, 54, 55, 56, 57, 58]"
4,35.500786,11.409627,4,59,02cf0ce1-699a-373b-86c0-eb6fd5f4697a,59,"[59, 60, 61, 62, 63, 64]"
...,...,...,...,...,...,...,...
3209,-13.251205,-72.100937,3209,130,ff78e1a3-6deb-34a4-9a1f-b85e34980f06,130,"[130, 131]"
3210,-11.547422,-73.086365,3210,132,ff78e1a3-6deb-34a4-9a1f-b85e34980f06,132,"[132, 133]"
3211,-13.253057,-72.100975,3211,134,ff78e1a3-6deb-34a4-9a1f-b85e34980f06,134,"[134, 135, 136, 137, 138, 139, 140, 141, 142, ..."
3212,-14.495268,-71.305946,3212,144,ff78e1a3-6deb-34a4-9a1f-b85e34980f06,144,"[144, 145, 146, 147, 148, 149]"


In [7]:
import plotly.express as px
fig = px.scatter(res, x= 'x', y = 'y', hover_data=['frame', "logId", "frames_list"], color = "logId")
fig.show()

In [10]:
res.drop(columns=["same_frame"], inplace = True)

In [11]:


jsonWriter = open("../public/tsne/projection.json", 'w')
jsonWriter.write(json.dumps(res.to_dict(orient="records")))
jsonWriter.close()